In [ ]:
import gradio as gr
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO

# Load models
yolo_model = YOLO("yolov8n.pt")
interpreter = tf.lite.Interpreter(model_path="/kaggle/input/model-cnn/model_cnn.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Class labels & prices
class_names = ["Ca hu kho", "Canh cai", "Canh chua", "Com trang", "Dau hu sot ca",
               "Ga chien", "Rau muong xao", "Thit kho", "Thit kho trung", "Trung chien"]
food_prices = dict(zip(class_names, [10000, 8000, 8000, 5000, 7000, 12000, 6000, 12000, 14000, 7000]))

def classify_image(image):
    detections = yolo_model(image)[0].boxes.data.cpu().numpy()
    predicted_classes, total_price = [], 0

    for x1, y1, x2, y2, score, class_id in detections:
        if score < 0.3: continue
        crop = image[int(y1):int(y2), int(x1):int(x2)]
        if crop.size == 0: continue
        input_data = np.expand_dims(cv2.resize(crop, (224, 224)).astype(np.float32) / 255.0, axis=0)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        pred_idx = int(np.argmax(interpreter.get_tensor(output_details[0]['index'])))
        label = class_names[pred_idx]
        predicted_classes.append(label)
        total_price += food_prices.get(label, 0)
        
    if not predicted_classes:
        return "Không phát hiện được món ăn", 0
    result_text = "\n".join([f"{food}: {food_prices[food]:,}đ" for food in predicted_classes])
    return result_text, f"{total_price:,}đ"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🍱 Nhận diện món ăn & Tính tiền")
    image_input = gr.Image(type="numpy", label="Tải ảnh mâm cơm")
    food_output = gr.Textbox(label="Các món ăn và giá", lines=10)
    total_output = gr.Textbox(label="Tổng tiền")
    gr.Button("Dự đoán và tính tiền").click(classify_image, inputs=image_input, outputs=[food_output, total_output])

demo.launch()